In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import measure
from skimage.measure import regionprops
from scipy import ndimage
import tifffile
import seaborn as sns
from scipy import stats
from pathlib import Path
import re
from google.colab import drive
# Mount Google Drive if you're running in Colab
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except:
    print("Not running in Colab or Drive already mounted")

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['figure.dpi'] = 100
plt.style.use('ggplot')

# Define your input and output paths
input_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Nuclei"  # Your specified folder
output_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3-x20/Overall/Nuclei"  # Your specified output folder

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Function to extract pressure information from filename
def extract_pressure_info(filename):
    """
    Extract pressure information (0Pa or 1.4Pa) from filename
    Examples: denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_filtered_mask
              denoised_0Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_filtered_mask
    """
    # Find pressure pattern in the filename (0Pa or 1.4Pa)
    pressure_match = re.search(r'(\d+\.?\d*Pa)', filename)

    if pressure_match:
        pressure = pressure_match.group(1)
        print(f"Found pressure in filename '{filename}': {pressure}")
        return pressure

    print(f"WARNING: Could not extract pressure from filename '{filename}'")
    return None

# Function to analyze nuclei masks
def analyze_nuclei_mask(mask_path):
    """Extract morphometric features focused on nuclei orientation and shape"""
    try:
        # Read mask
        mask = tifffile.imread(mask_path)

        # Ensure binary mask
        if mask.dtype != bool:
            mask = mask > 0

        # Label connected components
        labeled_mask, num_features = ndimage.label(mask)

        if num_features == 0:
            print(f"No features found in {mask_path}")
            return None

        # Calculate region properties
        regions = regionprops(labeled_mask)

        # Get pressure info from filename
        filename = os.path.basename(mask_path)
        pressure = extract_pressure_info(filename)

        if not pressure:
            print(f"Could not extract pressure info from {filename}")
            return None

        features_list = []

        # Extract features for each nucleus in the mask
        for i, region in enumerate(regions):
            # Skip very small regions (likely noise)
            if region.area < 10:
                continue

            # Calculate additional features
            # Aspect ratio = major_axis_length / minor_axis_length
            aspect_ratio = region.major_axis_length / region.minor_axis_length if region.minor_axis_length > 0 else 0

            # Flow direction is left-to-right, so we're interested in how aligned nuclei are with this direction
            # Convert orientation (-pi/2 to pi/2) to angle in degrees (0 to 180)
            # 0 or 180 degrees means aligned with flow, 90 degrees means perpendicular to flow
            orientation_rad = region.orientation

            # Convert to degrees (0-180 range)
            orientation_deg = np.degrees(orientation_rad)
            if orientation_deg < 0:
                orientation_deg += 180

            # Calculate alignment with flow (0 degrees or 180 degrees is perfect alignment)
            # This creates a value between 0 (aligned with flow) and 90 (perpendicular to flow)
            flow_alignment = min(orientation_deg, 180-orientation_deg)

            features = {
                'pressure': pressure,
                'nucleus_id': i,
                # Size features
                'area': region.area,
                'perimeter': region.perimeter,
                'equivalent_diameter': region.equivalent_diameter,
                # Shape features
                'major_axis_length': region.major_axis_length,
                'minor_axis_length': region.minor_axis_length,
                'aspect_ratio': aspect_ratio,
                'eccentricity': region.eccentricity,
                # Orientation features
                'orientation_degrees': orientation_deg,
                'flow_alignment_degrees': flow_alignment,
                # Other shape descriptors
                'solidity': region.solidity,
                # Centroid position
                'centroid_y': region.centroid[0],
                'centroid_x': region.centroid[1]
            }

            features_list.append(features)

        return features_list

    except Exception as e:
        print(f"Error analyzing mask {mask_path}: {e}")
        return None

# Main analysis function
def analyze_nuclei_morphology(input_dir, output_dir):
    """Analyze nuclei morphology with focus on orientation relative to flow direction"""
    print("Starting nuclei morphological analysis...")

    # Check if the input directory exists
    if not os.path.exists(input_dir):
        print(f"ERROR: Input directory does not exist: {input_dir}")
        return

    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")

    # Find all mask files (matching your nuclei file pattern)
    mask_files = []
    for file in os.listdir(input_dir):
        # MODIFIED: Updated pattern to match the actual files in the directory
        if file.endswith('.tif') and 'Cadherins_filtered_mask' in file:
            mask_files.append(os.path.join(input_dir, file))

    # Print the files found to help with debugging
    print("Files found:")
    for file in mask_files:
        print(f"  - {os.path.basename(file)}")

    print(f"Found {len(mask_files)} nuclei mask files")

    # Extract features from each mask
    all_features = []
    for file in mask_files:
        features = analyze_nuclei_mask(file)
        if features:
            all_features.extend(features)

    # Convert to DataFrame
    if not all_features:
        print("No features could be extracted. Check your mask files.")
        return

    df = pd.DataFrame(all_features)

    # Save raw data
    features_csv = os.path.join(output_dir, "nuclei_morphological_features.csv")
    df.to_csv(features_csv, index=False)
    print(f"Saved features to {features_csv}")

    # Get unique pressures for comparison
    pressures = df['pressure'].unique()
    print(f"Comparing pressures: {pressures}")

    # Features to analyze
    features_to_compare = [
        'area',
        'aspect_ratio',
        'flow_alignment_degrees',
        'major_axis_length',
        'eccentricity'
    ]

    # Create violin plots for each feature to compare between pressure conditions
    for feature in features_to_compare:
        plt.figure(figsize=(10, 6))
        # Create violin plot with inner points showing individual data points
        ax = sns.violinplot(x='pressure', y=feature, data=df, inner='point', cut=0)

        # Add means as markers with different color
        means = df.groupby('pressure')[feature].mean()
        for i, pressure in enumerate(means.index):
            plt.plot(i, means[pressure], 'o', color='red', markersize=10,
                    markeredgecolor='black', markeredgewidth=1.5, label='Mean' if i == 0 else "")

        # Add legend for the mean marker (only once)
        plt.legend(loc='upper right')

        # Add statistical comparison if we have two pressure groups
        if len(pressures) == 2:
            # Perform t-test between groups
            group1 = df[df['pressure'] == pressures[0]][feature].dropna()
            group2 = df[df['pressure'] == pressures[1]][feature].dropna()

            if len(group1) > 0 and len(group2) > 0:
                stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)

                # Add significance annotation
                sig_text = f'p = {p_value:.3f}'
                if p_value < 0.001:
                    sig_text += ' ***'
                elif p_value < 0.01:
                    sig_text += ' **'
                elif p_value < 0.05:
                    sig_text += ' *'

                plt.annotate(sig_text, xy=(0.5, 0.95), xycoords='axes fraction',
                            ha='center', va='center',
                            bbox=dict(boxstyle='round', fc='white', alpha=0.7))

        plt.title(f'Nuclei {feature.replace("_", " ").title()} by Pressure')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"nuclei_{feature}_violinplot.png"), dpi=300)
        plt.close()

    # Create polar histogram of nuclei orientations
    plt.figure(figsize=(12, 10))

    # Subplots for each pressure condition
    fig, axes = plt.subplots(1, len(pressures), figsize=(7*len(pressures), 6), subplot_kw={'projection': 'polar'})

    # Handle case with only one pressure condition
    if len(pressures) == 1:
        axes = [axes]

    for i, pressure in enumerate(pressures):
        pressure_data = df[df['pressure'] == pressure]

        # Convert degrees to radians for polar plot
        orientation_rad = np.deg2rad(pressure_data['orientation_degrees'])

        # Create polar histogram
        axes[i].hist(orientation_rad, bins=36, alpha=0.7)
        axes[i].set_title(f'Nuclei Orientation - {pressure}')

        # Mark flow direction (0° and 180°)
        axes[i].annotate('Flow →', xy=(0, 0.9), xytext=(0, 0.9), xycoords='data',
                        textcoords='data', ha='center', va='center',
                        bbox=dict(boxstyle='round', fc='white', alpha=0.7))
        axes[i].annotate('Flow ←', xy=(np.pi, 0.9), xytext=(np.pi, 0.9), xycoords='data',
                        textcoords='data', ha='center', va='center',
                        bbox=dict(boxstyle='round', fc='white', alpha=0.7))

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "nuclei_orientation_polar_histogram.png"), dpi=300)
    plt.close()

    # Create scatter plots for area vs aspect ratio
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='area', y='aspect_ratio', hue='pressure', data=df, alpha=0.6, palette='viridis')

    # Add regression lines for each pressure
    for pressure in pressures:
        pressure_data = df[df['pressure'] == pressure]
        sns.regplot(x='area', y='aspect_ratio', data=pressure_data, scatter=False, label=f'{pressure} trend')

    plt.title('Nuclei: Aspect Ratio vs Area')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "nuclei_aspect_ratio_vs_area.png"), dpi=300)
    plt.close()

    # Create scatter plots for flow alignment vs aspect ratio
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='flow_alignment_degrees', y='aspect_ratio', hue='pressure', data=df, alpha=0.6, palette='viridis')

    # Add regression lines for each pressure
    for pressure in pressures:
        pressure_data = df[df['pressure'] == pressure]
        sns.regplot(x='flow_alignment_degrees', y='aspect_ratio', data=pressure_data, scatter=False, label=f'{pressure} trend')

    plt.title('Nuclei: Aspect Ratio vs Flow Alignment')
    plt.xlabel('Flow Alignment (degrees from flow direction)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "nuclei_aspect_ratio_vs_flow_alignment.png"), dpi=300)
    plt.close()

    # Visualize nuclei position and orientation on a 2D plot
    for pressure in pressures:
        pressure_data = df[df['pressure'] == pressure]

        plt.figure(figsize=(12, 10))

        # Plot nuclei positions
        plt.scatter(pressure_data['centroid_x'], pressure_data['centroid_y'],
                  s=pressure_data['area']/10, alpha=0.5, label='Nuclei')

        # Add orientation lines
        for _, nucleus in pressure_data.iterrows():
            # Calculate line endpoints based on orientation
            length = nucleus['major_axis_length'] / 2
            angle_rad = np.deg2rad(nucleus['orientation_degrees'])
            dx = length * np.cos(angle_rad)
            dy = length * np.sin(angle_rad)

            # Draw line through centroid showing orientation
            plt.plot([nucleus['centroid_x'] - dx, nucleus['centroid_x'] + dx],
                   [nucleus['centroid_y'] - dy, nucleus['centroid_y'] + dy],
                   'r-', alpha=0.3)

        # Add flow direction indicator
        plt.arrow(plt.xlim()[0] + 50, plt.ylim()[1] - 50, 100, 0, head_width=20,
                head_length=20, fc='blue', ec='blue', label='Flow Direction')
        plt.text(plt.xlim()[0] + 100, plt.ylim()[1] - 30, 'Flow Direction', color='blue')

        plt.title(f'Nuclei Positions and Orientations - {pressure}')
        plt.xlabel('X position')
        plt.ylabel('Y position')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"nuclei_positions_{pressure}.png"), dpi=300)
        plt.close()

    # Create summary statistics
    summary = df.groupby(['pressure']).agg({
        'area': ['mean', 'std', 'median', 'count'],
        'aspect_ratio': ['mean', 'std', 'median'],
        'flow_alignment_degrees': ['mean', 'std', 'median'],
        'eccentricity': ['mean', 'std', 'median'],
    }).reset_index()

    summary.to_csv(os.path.join(output_dir, "nuclei_summary_statistics.csv"))

    print("Nuclei morphological analysis complete! Results saved to:", output_dir)

# Run the analysis
if __name__ == "__main__":
    # Print the current working directory for debugging
    print(f"Current working directory: {os.getcwd()}")

    # List all files in the input directory to confirm access
    try:
        all_files = os.listdir(input_dir)
        print(f"All files in input directory ({len(all_files)} files):")
        for file in all_files[:10]:  # Print first 10 files to avoid overwhelming output
            print(f"  - {file}")
        if len(all_files) > 10:
            print(f"  ... and {len(all_files) - 10} more files")
    except Exception as e:
        print(f"Error accessing input directory: {e}")

    # Run the analysis function
    analyze_nuclei_morphology(input_dir, output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!
Current working directory: /content
All files in input directory (8 files):
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_filtered_mask.tif
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_Cadherins_filtered_mask.tif
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq003_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq001_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq002_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq003_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq004_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_filtered_mask.tif
Starting nuclei morphological analysis...
Input directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Nuclei
Out

<Figure size 1200x1000 with 0 Axes>

In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import measure
from skimage.measure import regionprops
from scipy import ndimage
import tifffile
import seaborn as sns
from scipy import stats
from pathlib import Path
import re
from google.colab import drive
# Mount Google Drive if you're running in Colab
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except:
    print("Not running in Colab or Drive already mounted")

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['figure.dpi'] = 100
plt.style.use('ggplot')

# Define your input and output paths
input_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Nuclei"  # Your specified folder
output_dir = "/content/drive/MyDrive/knowledge/University/Master/Thesis/Analysis/flow3-x20/Overall/Nuclei"  # Your specified output folder

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Function to extract pressure information from filename
def extract_pressure_info(filename):
    """
    Extract pressure information (0Pa or 1.4Pa) from filename
    Examples: denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_filtered_mask
              denoised_0Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_filtered_mask
    """
    # Find pressure pattern in the filename (0Pa or 1.4Pa)
    pressure_match = re.search(r'(\d+\.?\d*Pa)', filename)

    if pressure_match:
        pressure = pressure_match.group(1)
        print(f"Found pressure in filename '{filename}': {pressure}")
        return pressure

    print(f"WARNING: Could not extract pressure from filename '{filename}'")
    return None

# Function to analyze nuclei masks
def analyze_nuclei_mask(mask_path):
    """Extract morphometric features focused on nuclei orientation and shape"""
    try:
        # Read mask
        mask = tifffile.imread(mask_path)

        # Ensure binary mask
        if mask.dtype != bool:
            mask = mask > 0

        # Label connected components
        labeled_mask, num_features = ndimage.label(mask)

        if num_features == 0:
            print(f"No features found in {mask_path}")
            return None

        # Calculate region properties
        regions = regionprops(labeled_mask)

        # Get pressure info from filename
        filename = os.path.basename(mask_path)
        pressure = extract_pressure_info(filename)

        if not pressure:
            print(f"Could not extract pressure info from {filename}")
            return None

        features_list = []

        # Extract features for each nucleus in the mask
        for i, region in enumerate(regions):
            # Skip very small regions (likely noise)
            if region.area < 10:
                continue

            # Calculate additional features
            # Aspect ratio = major_axis_length / minor_axis_length
            aspect_ratio = region.major_axis_length / region.minor_axis_length if region.minor_axis_length > 0 else 0

            # Flow direction is left-to-right, so we're interested in how aligned nuclei are with this direction
            # Convert orientation (-pi/2 to pi/2) to angle in degrees (0 to 180)
            # 0 or 180 degrees means aligned with flow, 90 degrees means perpendicular to flow
            orientation_rad = region.orientation

            # Convert to degrees (0-180 range)
            orientation_deg = np.degrees(orientation_rad)
            if orientation_deg < 0:
                orientation_deg += 180

            # Calculate alignment with flow (0 degrees or 180 degrees is perfect alignment)
            # This creates a value between 0 (aligned with flow) and 90 (perpendicular to flow)
            flow_alignment = min(orientation_deg, 180-orientation_deg)

            features = {
                'pressure': pressure,
                'nucleus_id': i,
                # Size features
                'area': region.area,
                'perimeter': region.perimeter,
                'equivalent_diameter': region.equivalent_diameter,
                # Shape features
                'major_axis_length': region.major_axis_length,
                'minor_axis_length': region.minor_axis_length,
                'aspect_ratio': aspect_ratio,
                'eccentricity': region.eccentricity,
                # Orientation features
                'orientation_degrees': orientation_deg,
                'flow_alignment_degrees': flow_alignment,
                # Other shape descriptors
                'solidity': region.solidity,
                # Centroid position
                'centroid_y': region.centroid[0],
                'centroid_x': region.centroid[1]
            }

            features_list.append(features)

        return features_list

    except Exception as e:
        print(f"Error analyzing mask {mask_path}: {e}")
        return None

# Main analysis function
def analyze_nuclei_morphology(input_dir, output_dir):
    """Analyze nuclei morphology with focus on orientation relative to flow direction"""
    print("Starting nuclei morphological analysis...")

    # Check if the input directory exists
    if not os.path.exists(input_dir):
        print(f"ERROR: Input directory does not exist: {input_dir}")
        return

    print(f"Input directory: {input_dir}")
    print(f"Output directory: {output_dir}")

    # Find all mask files (matching your nuclei file pattern)
    mask_files = []
    for file in os.listdir(input_dir):
        # MODIFIED: Updated pattern to match the actual files in the directory
        if file.endswith('.tif') and 'Cadherins_filtered_mask' in file:
            mask_files.append(os.path.join(input_dir, file))

    # Print the files found to help with debugging
    print("Files found:")
    for file in mask_files:
        print(f"  - {os.path.basename(file)}")

    print(f"Found {len(mask_files)} nuclei mask files")

    # Extract features from each mask
    all_features = []
    for file in mask_files:
        features = analyze_nuclei_mask(file)
        if features:
            all_features.extend(features)

    # Convert to DataFrame
    if not all_features:
        print("No features could be extracted. Check your mask files.")
        return

    df = pd.DataFrame(all_features)

    # Save raw data
    features_csv = os.path.join(output_dir, "nuclei_morphological_features.csv")
    df.to_csv(features_csv, index=False)
    print(f"Saved features to {features_csv}")

    # Get unique pressures for comparison
    pressures = df['pressure'].unique()
    print(f"Comparing pressures: {pressures}")

    # Features to analyze
    features_to_compare = [
        'area',
        'aspect_ratio',
        'flow_alignment_degrees',
        'major_axis_length',
        'eccentricity'
    ]

    # Create violin plots for each feature to compare between pressure conditions
    for feature in features_to_compare:
        plt.figure(figsize=(10, 6))
        # Create violin plot with inner points showing individual data points
        ax = sns.violinplot(x='pressure', y=feature, data=df, inner='point', cut=0)

        # Add means as markers with horizontal lines for better visualization
        means = df.groupby('pressure')[feature].mean()

        # For each pressure group
        for i, pressure in enumerate(means.index):
            mean_value = means[pressure]

            # Add horizontal line across the violin at the mean height
            violin_width = 0.3  # Half-width of violin plot
            plt.hlines(y=mean_value, xmin=i-violin_width, xmax=i+violin_width,
                      color='red', linestyle='-', linewidth=2, label='Mean' if i == 0 else "")

            # Add the mean value as text next to the line
            plt.text(i+violin_width+0.05, mean_value, f"Mean: {mean_value:.2f}",
                   verticalalignment='center', color='red', fontweight='bold')

            # Also keep the point marker
            plt.plot(i, mean_value, 'o', color='red', markersize=7,
                   markeredgecolor='black', markeredgewidth=1.5)

        # Add legend for the mean marker (only once)
        handles, labels = plt.gca().get_legend_handles_labels()
        if len(handles) > 0:
            plt.legend(loc='upper right')

        # Add statistical comparison if we have two pressure groups
        if len(pressures) == 2:
            # Perform t-test between groups
            group1 = df[df['pressure'] == pressures[0]][feature].dropna()
            group2 = df[df['pressure'] == pressures[1]][feature].dropna()

            if len(group1) > 0 and len(group2) > 0:
                stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)

                # Add significance annotation
                sig_text = f'p = {p_value:.3f}'
                if p_value < 0.001:
                    sig_text += ' ***'
                elif p_value < 0.01:
                    sig_text += ' **'
                elif p_value < 0.05:
                    sig_text += ' *'

                plt.annotate(sig_text, xy=(0.5, 0.95), xycoords='axes fraction',
                            ha='center', va='center',
                            bbox=dict(boxstyle='round', fc='white', alpha=0.7))

        plt.title(f'Nuclei {feature.replace("_", " ").title()} by Pressure')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"nuclei_{feature}_violinplot.png"), dpi=300)
        plt.close()

    # Create polar histogram of nuclei orientations
    plt.figure(figsize=(12, 10))

    # Subplots for each pressure condition
    fig, axes = plt.subplots(1, len(pressures), figsize=(7*len(pressures), 6), subplot_kw={'projection': 'polar'})

    # Handle case with only one pressure condition
    if len(pressures) == 1:
        axes = [axes]

    for i, pressure in enumerate(pressures):
        pressure_data = df[df['pressure'] == pressure]

        # Convert degrees to radians for polar plot
        orientation_rad = np.deg2rad(pressure_data['orientation_degrees'])

        # Create polar histogram
        axes[i].hist(orientation_rad, bins=36, alpha=0.7)
        axes[i].set_title(f'Nuclei Orientation - {pressure}')

        # Mark flow direction (0° and 180°)
        axes[i].annotate('Flow →', xy=(0, 0.9), xytext=(0, 0.9), xycoords='data',
                        textcoords='data', ha='center', va='center',
                        bbox=dict(boxstyle='round', fc='white', alpha=0.7))
        axes[i].annotate('Flow ←', xy=(np.pi, 0.9), xytext=(np.pi, 0.9), xycoords='data',
                        textcoords='data', ha='center', va='center',
                        bbox=dict(boxstyle='round', fc='white', alpha=0.7))

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "nuclei_orientation_polar_histogram.png"), dpi=300)
    plt.close()

    # Create scatter plots for area vs aspect ratio
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='area', y='aspect_ratio', hue='pressure', data=df, alpha=0.6, palette='viridis')

    # Add regression lines for each pressure
    for pressure in pressures:
        pressure_data = df[df['pressure'] == pressure]
        sns.regplot(x='area', y='aspect_ratio', data=pressure_data, scatter=False, label=f'{pressure} trend')

    plt.title('Nuclei: Aspect Ratio vs Area')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "nuclei_aspect_ratio_vs_area.png"), dpi=300)
    plt.close()

    # Create scatter plots for flow alignment vs aspect ratio
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='flow_alignment_degrees', y='aspect_ratio', hue='pressure', data=df, alpha=0.6, palette='viridis')

    # Add regression lines for each pressure
    for pressure in pressures:
        pressure_data = df[df['pressure'] == pressure]
        sns.regplot(x='flow_alignment_degrees', y='aspect_ratio', data=pressure_data, scatter=False, label=f'{pressure} trend')

    plt.title('Nuclei: Aspect Ratio vs Flow Alignment')
    plt.xlabel('Flow Alignment (degrees from flow direction)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "nuclei_aspect_ratio_vs_flow_alignment.png"), dpi=300)
    plt.close()

    # Visualize nuclei position and orientation on a 2D plot
    for pressure in pressures:
        pressure_data = df[df['pressure'] == pressure]

        plt.figure(figsize=(12, 10))

        # Plot nuclei positions
        plt.scatter(pressure_data['centroid_x'], pressure_data['centroid_y'],
                  s=pressure_data['area']/10, alpha=0.5, label='Nuclei')

        # Add orientation lines
        for _, nucleus in pressure_data.iterrows():
            # Calculate line endpoints based on orientation
            length = nucleus['major_axis_length'] / 2
            angle_rad = np.deg2rad(nucleus['orientation_degrees'])
            dx = length * np.cos(angle_rad)
            dy = length * np.sin(angle_rad)

            # Draw line through centroid showing orientation
            plt.plot([nucleus['centroid_x'] - dx, nucleus['centroid_x'] + dx],
                   [nucleus['centroid_y'] - dy, nucleus['centroid_y'] + dy],
                   'r-', alpha=0.3)

        # Add flow direction indicator
        plt.arrow(plt.xlim()[0] + 50, plt.ylim()[1] - 50, 100, 0, head_width=20,
                head_length=20, fc='blue', ec='blue', label='Flow Direction')
        plt.text(plt.xlim()[0] + 100, plt.ylim()[1] - 30, 'Flow Direction', color='blue')

        plt.title(f'Nuclei Positions and Orientations - {pressure}')
        plt.xlabel('X position')
        plt.ylabel('Y position')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"nuclei_positions_{pressure}.png"), dpi=300)
        plt.close()

    # Create summary statistics
    summary = df.groupby(['pressure']).agg({
        'area': ['mean', 'std', 'median', 'count'],
        'aspect_ratio': ['mean', 'std', 'median'],
        'flow_alignment_degrees': ['mean', 'std', 'median'],
        'eccentricity': ['mean', 'std', 'median'],
    }).reset_index()

    summary.to_csv(os.path.join(output_dir, "nuclei_summary_statistics.csv"))

    print("Nuclei morphological analysis complete! Results saved to:", output_dir)

# Run the analysis
if __name__ == "__main__":
    # Print the current working directory for debugging
    print(f"Current working directory: {os.getcwd()}")

    # List all files in the input directory to confirm access
    try:
        all_files = os.listdir(input_dir)
        print(f"All files in input directory ({len(all_files)} files):")
        for file in all_files[:10]:  # Print first 10 files to avoid overwhelming output
            print(f"  - {file}")
        if len(all_files) > 10:
            print(f"  ... and {len(all_files) - 10} more files")
    except Exception as e:
        print(f"Error accessing input directory: {e}")

    # Run the analysis function
    analyze_nuclei_morphology(input_dir, output_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!
Current working directory: /content
All files in input directory (8 files):
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq001_Cadherins_filtered_mask.tif
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq002_Cadherins_filtered_mask.tif
  - denoised_0Pa_U_05mar19_20x_L2RA_Flat_seq003_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq001_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq002_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq003_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq004_Cadherins_filtered_mask.tif
  - denoised_1.4Pa_U_05mar19_20x_L2R_Flat_seq005_Cadherins_filtered_mask.tif
Starting nuclei morphological analysis...
Input directory: /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/flow3-x20/Nuclei
Out

<Figure size 1200x1000 with 0 Axes>